In [17]:
import os
from collections import Counter

import dask.dataframe as dd
import pandas as pd
from matplotlib import pyplot as plt
from web3 import Web3
import pandas as pd

from graph_reinforcement_learning_using_blockchain_data import config

config.load_dotenv()


True

In [2]:
def parse_logs(s: str):
    safe_globals = {
        "__builtins__": {},
        "AttributeDict": lambda d: d,  # return the dict directly
        "HexBytes": lambda x: x,  # return the string argument
    }
    try:
        return eval(s, safe_globals)
    except Exception as e:
        print("Error parsing logs:", e)
        return []


TODO: Download whole transaction receipt for arbs

In [ ]:
# dd_arbs = dd.read_csv(config.PROCESSED_DATA_DIR / "flashbots" / "Q2_2023" / "arbitrages_with_logs.csv",
#                       dtype={'end_amount': 'object',
#                              'error': 'object',
#                              'profit_amount': 'object',
#                              'start_amount': 'object',
#                              },
#                       )
# dd_arbs["logs"] = dd_arbs["logs"].apply(parse_logs, meta=("logs", "object"))

In [9]:
dd_class0 = dd.read_csv(config.PROCESSED_DATA_DIR / "flashbots" / "Q2_2023" / "class0_with_logs.csv")
dd_class0["receipt"] = dd_class0["receipt"].apply(parse_logs, meta=("logs", "object"))
dd_class0["logs"] = dd_class0["receipt"].apply(lambda x: x["logs"], meta=("logs", "object"))

In [29]:
print(dd_class0["receipt"].head())

0    {'transactionHash': '0x8e90ecb85b61af368db0154...
1    {'transactionHash': '0xd83c25449d0766a18945f10...
2    {'transactionHash': '0x8b434ba680a93cc0e85a7c4...
3    {'transactionHash': '0x8d3b8e46a66a909643f2bb7...
4    {'transactionHash': '0x25908bad48b2353a1da142d...
Name: receipt, dtype: object


In [10]:
# See notebook 0.03-lte-signatures-arbitrages_with_logs.ipynb
top_class0_signatures = ['0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef',
                         '0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925',
                         '0x1c411e9a96e071241c2f21f7726b17ae89e3cab4c78be50e062b03a9fffbbad1',
                         '0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822',
                         '0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c',
                         '0x7fcf532c15f0a6db0bd6d0e038bea71d30d808c7d98cb3bf7268a95bf5081b65',
                         '0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67',
                         '0xe9149e1b5059238baed02fa659dbf4bd932fbcf760a431330df4d934bc942f37',
                         '0x9d9af8e38d66c62e2c12f0225249fd9d721c54b83f48d9352c97c6cacdcb6f31',
                         '0x17307eab39ab6107e8899845ad3d59bd9653f200f220920489ca2b5937696c31']

top_class1_signatures = ['0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef',
                          '0x1c411e9a96e071241c2f21f7726b17ae89e3cab4c78be50e062b03a9fffbbad1',
                          '0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822',
                          '0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67',
                          '0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925',
                          '0x7fcf532c15f0a6db0bd6d0e038bea71d30d808c7d98cb3bf7268a95bf5081b65',
                          '0x19b47279256b2a23a1665c810c8d55a1758940ee09377d4f8d26497a3577dc83',
                          '0x908fb5ee8f16c6bc9bc3690973819f32a4d4b10188134543c88706e0e1d43378',
                          '0xddac40937f35385a34f721af292e5a83fc5b840f722bff57c2fc71adba708c48',
                          '0x8201aa3f00000000000000000000000000000000000000000000000000000000']

In [39]:
def extract_receipt_features(tx_receipt, top_normal_signatures, top_arb_signatures):
    features = {}
    # Basic receipt-level features
    features["transactionHash"] = tx_receipt.get("transactionHash")
    features["blockNumber"] = tx_receipt.get("blockNumber", 0)
    features["gasUsed"] = tx_receipt.get("gasUsed", 0)
    features["cumulativeGasUsed"] = tx_receipt.get("cumulativeGasUsed", 0)
    features["transactionIndex"] = tx_receipt.get("transactionIndex", 0)
    features["effectiveGasPrice"] = tx_receipt.get("effectiveGasPrice", 0)
    features["status"] = tx_receipt.get("status", 0)
    features["fee"] = features["gasUsed"] * features["effectiveGasPrice"]

    # Include the main from and to addresses
    features["from"] = tx_receipt.get("from", None)
    features["to"] = tx_receipt.get("to", None)
    
    # Log-based features
    logs = tx_receipt.get("logs", [])
    features["num_logs"] = len(logs)
    
    # Count occurrences of events by signature (first topic)
    event_counts = {}
    for log in logs:
        topics = log.get("topics", [])
        if topics:
            # Convert the signature to a hex string if necessary
            sig = topics[0].hex() if hasattr(topics[0], "hex") else str(topics[0])
            event_counts[sig] = event_counts.get(sig, 0) + 1

    # Add dummy features for top signatures (normal and arb)
    for sig in top_normal_signatures:
        features[f"dummy_normal_{sig[:10]}"] = 1 if sig in event_counts else 0
    for sig in top_arb_signatures:
        features[f"dummy_arb_{sig[:10]}"] = 1 if sig in event_counts else 0

    return features

In [40]:
features_dd = dd_class0["receipt"].apply(
    lambda row: extract_receipt_features(row, top_class0_signatures, top_class1_signatures),
    axis=0,
    meta=('features', 'object')
)

In [41]:
features = features_dd.compute()

In [42]:
features_df = pd.DataFrame(list(features))

In [43]:
features_df.head()

,transactionHash,blockNumber,gasUsed,cumulativeGasUsed,transactionIndex,effectiveGasPrice,status,fee,from,to,...,dummy_arb_0xddf252ad,dummy_arb_0x1c411e9a,dummy_arb_0xd78ad95f,dummy_arb_0xc42079f9,dummy_arb_0x8c5be1e5,dummy_arb_0x7fcf532c,dummy_arb_0x19b47279,dummy_arb_0x908fb5ee,dummy_arb_0xddac4093,dummy_arb_0x8201aa3f
0,0x8e90ecb85b61af368db0154830453552f0c27e0e1812...,17005294,153529,10750253,111,19150328915,1,2940130847991035,0x832aC338d06f5A438f3340B724c08C44e744e9c2,0x00000000000001ad428e4906aE43D8F9852d0dD6,...,1,0,0,0,1,0,0,0,0,0
1,0xd83c25449d0766a18945f10aff7912f2ff492f0234a8...,16962424,123497,229900,1,139502435999,1,17228132338568503,0x080086911D8c78008800FAE75871a657b77d0082,0x0000E0Ca771e21bD00057F54A68C30D400000000,...,1,1,1,0,0,0,0,0,0,0
2,0x8b434ba680a93cc0e85a7c431e5b35a8efb60ef05c24...,16993741,21000,15838026,183,22098066562,1,464059397802000,0xE991Dc594Ea50b7B83be1B0a44Fd138582c04383,0xD8A5859Ce6f265D376922Dc27f70b804cc65f49E,...,0,0,0,0,0,0,0,0,0,0
3,0x8d3b8e46a66a909643f2bb7497b18c954fc22fa5bd5c...,16972733,46817,14885776,165,18825121161,1,881335697394537,0xCA8ba7d3f798d1872A8C05c5358a1b24a4d3Ac64,0x2b591e99afE9f32eAA6214f7B7629768c40Eeb39,...,1,0,0,0,0,0,0,0,0,0
4,0x25908bad48b2353a1da142da6f0a19c015dfbac5245f...,17028487,32215,3160909,48,19660583885,1,633365709855275,0x8FEf490D614fCe8B93Bd6F28835dD35a8B3229a9,0xBA50933C268F567BDC86E1aC131BE072C6B0b71a,...,1,0,0,0,0,0,0,0,0,0
